# Features Games

In [1]:
import os
import re

project_path = re.findall(
    pattern=r".+monday-vip-consulting",
    string=os.path.abspath(os.getcwd()),
)[0]
os.chdir(os.path.join(project_path))
%load_ext jupyter_black

In [2]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px


from src.utils.pather import Pather
from src.data.make_features_dataset import FeaturesDataset

pather = Pather()

### Accounts Features

In [3]:
accounts = pd.read_csv(pather.interim_accounts)
accounts.shape

(716828, 20)

In [4]:
accounts.head(10)

,account_id,account_name,created_at,trial_start,time_diff,region,country,paying,has_logo,device,os,browser,collection_21_days,payment_currency,max_team_size,min_team_size,industry,team_size,lead_score,plan_id
0,235592,"Lopez, Evans and Mitchell",2019-01-29 10:24:05,2019-01-29 10:24:05,-2.0,Sao Paulo,BR,0,1,desktop,windows,chrome,0,USD,5.0,2.0,Creative,2-5,0,NaN
1,1240640,"Berger, Roberts and Martin",2019-06-05 20:19:01,2019-06-05 20:19:01,2.0,Province of North West,ZA,0,1,mobile,android,chrome,0,USD,5.0,2.0,NaN,2-5,0,NaN
2,553815,Burns-Long,2019-03-03 10:28:26,2019-03-03 10:28:26,-3.0,Sao Paulo,BR,0,1,mobile,ios,generic browser,0,USD,5.0,2.0,NaN,NaN,0,NaN
3,93251,Wilson Ltd,2019-01-13 15:54:58,2019-01-13 15:54:58,-6.0,Yucatan,MX,0,1,desktop,windows,chrome,0,USD,1.0,1.0,Administrative,1-1,0,NaN
4,873838,Sanchez-Mata,2019-04-12 05:31:20,2019-04-12 05:31:20,3.0,New Jersey,US,0,1,desktop,windows,chrome,0,USD,10.0,6.0,Administrative,6-10,0,NaN
5,817929,"Avila, Gay and Fisher",2019-04-05 04:14:15,2019-04-05 04:14:15,-5.0,Tamaulipas,MX,0,1,mobile,ios,generic browser,0,USD,1.0,1.0,NaN,NaN,0,NaN
6,881319,"Guzman, Smith and King",2019-04-13 09:42:07,2019-04-13 09:42:07,NaN,Washington,US,0,1,mobile,android,NaN,0,USD,NaN,NaN,NaN,NaN,0,NaN
7,996197,"Lee, Lee and Johnson",2019-05-01 21:52:08,2019-05-01 21:52:08,-3.0,Departamento de Montevideo,UY,0,1,desktop,windows,chrome,0,USD,15.0,11.0,NaN,NaN,0,NaN
8,1244051,Vasquez-Alexander,2019-06-06 06:39:17,2019-06-06 06:39:17,12.0,Wellington,NZ,0,1,desktop,windows,chrome,0,USD,25.0,16.0,Other,16-25,0,NaN
9,1309864,Young-Harris,2019-06-13 16:55:44,2019-06-13 16:55:44,-4.0,Massachusetts,US,0,1,mobile,ios,safari,0,USD,5.0,2.0,NaN,2-5,0,NaN


In [5]:
accounts.nunique()

account_id            716828
account_name          306579
created_at            697378
trial_start           698229
time_diff                 40
region                  3180
country                  223
paying                     2
has_logo                   1
device                     3
os                         7
browser                   12
collection_21_days      1159
payment_currency           8
max_team_size             12
min_team_size             12
industry                  32
team_size                 18
lead_score                 2
plan_id                  104
dtype: int64

In [6]:
features = [
    "account_id",
    "paying",
    "collection_21_days",
    "max_team_size",
    "min_team_size",
    "industry",
    "payment_currency",
    "lead_score",
]
account_features = (
    accounts[features]
    .copy()
    .set_index("account_id")
    .astype(
        {
            "industry": "category",
            "payment_currency": "category",
            # "region": "category",
            # "country": "category",
        }
    )
)

In [7]:
account_features

,paying,collection_21_days,max_team_size,min_team_size,industry,payment_currency,lead_score
account_id,,,,,,,
235592,0,0,5.0,2.0,Creative,USD,0
1240640,0,0,5.0,2.0,NaN,USD,0
553815,0,0,5.0,2.0,NaN,USD,0
93251,0,0,1.0,1.0,Administrative,USD,0
873838,0,0,10.0,6.0,Administrative,USD,0
...,...,...,...,...,...,...,...
1338612,0,0,NaN,NaN,NaN,USD,0
60593,0,0,1.0,1.0,Product Management,USD,0
980862,0,0,5.0,2.0,NaN,CAD,0


## Users Features

In [8]:
users = pd.read_csv(pather.interim_users)
users.shape

(1119360, 20)

In [9]:
users_features = (
    users.groupby("account_id")
    .agg(
        {
            "user_id": "nunique",
            "is_admin": "sum",
            "pending": "sum",
            "enabled": "sum",
        }
    )
    .rename(
        columns={
            "user_id": "registered_users",
            "is_admin": "number_of_admins",
            "pending": "number_of_pending_users",
            "enabled": "number_of_enabled_users",
        }
    )
    # .sort_values("registered_users", ascending=False)
)
users_features

,registered_users,number_of_admins,number_of_pending_users,number_of_enabled_users
account_id,,,,
4,1,1,0,1
6,1,1,0,1
7,1,1,0,1
8,1,1,0,1
9,1,1,0,1
...,...,...,...,...
1433650,1,1,0,1
1433651,1,1,0,1
1433653,1,0,1,1


In [10]:
users.nunique()

account_id           716099
user_id             1119360
email                854344
name                 299700
created_at          1038386
is_admin                  2
pending                   2
enabled                   2
became_active_at     892330
time_diff                40
city                  45457
region                 3311
country                 225
serial_number           227
has_photo                 1
device                    3
os                        7
browser                  12
has_phone                 1
lead_score                2
dtype: int64

In [11]:
users[users["account_id"] == 602168]

,account_id,user_id,email,name,created_at,is_admin,pending,enabled,became_active_at,time_diff,city,region,country,serial_number,has_photo,device,os,browser,has_phone,lead_score
471684,602168,945303,joannaharris@white-oneal.com,Caleb Singleton,2019-03-08 16:31:11,1,0,1,2019-03-08 16:33:01,-6.0,Cibolo,Texas,US,1,1,desktop,windows,chrome,1,1
471685,602168,945304,kayladoyle@white-oneal.com,Deanna Hernandez,2019-03-08 16:33:29,1,0,1,2019-03-08 16:37:58,-5.0,San Antonio,Texas,US,2,1,desktop,windows,chrome,1,1
471686,602168,945305,reidtheresa@white-oneal.com,Jessica Wolf,2019-03-08 19:18:44,0,0,1,2019-03-08 19:21:16,-5.0,Cibolo,Texas,US,3,1,desktop,windows,internet explorer,1,1
471687,602168,945306,johnsonfrank@white-oneal.com,Christopher Newman,2019-03-08 19:21:04,0,0,1,2019-03-08 19:23:21,-5.0,Dallas,Texas,US,4,1,desktop,windows,internet explorer,1,1
471688,602168,945307,longbonnie@white-oneal.com,Kelly Hall,2019-03-08 21:22:14,0,0,1,2019-03-08 21:26:33,-5.0,Austin,Texas,US,5,1,mobile,ios,safari,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471868,602168,945487,mark14@white-oneal.com,Michael Henson,2019-07-29 20:53:24,0,0,1,2019-07-29 21:35:12,-5.0,Irving,Texas,US,185,1,desktop,windows,internet explorer,1,1
471869,602168,945488,michelle62@white-oneal.com,Tina Barker,2019-07-30 19:41:06,0,0,1,2019-07-30 19:41:39,-5.0,San Antonio,Texas,US,186,1,desktop,windows,internet explorer,1,1
471870,602168,945489,chall@white-oneal.com,Eric Carey,2019-07-31 15:06:44,0,0,1,2019-07-31 15:08:23,-5.0,San Antonio,Texas,US,187,1,desktop,windows,internet explorer,1,1
471871,602168,945490,brownmark@white-oneal.com,Jeffrey Powell,2019-08-01 19:16:38,0,0,1,2019-08-01 19:24:08,NaN,Katy,Texas,US,188,1,mobile,ios,safari,1,1


# All Features

In [12]:
features_dataseter = FeaturesDataset()
accounts_features = features_dataseter._create_accounts_features()
events_features = features_dataseter._create_events_features()
users_features = features_dataseter._create_users_features()

Loading Data...


In [13]:
accounts_ids_not_in_users = accounts_features[
    ~accounts_features.index.isin(users_features.index)
]

In [14]:
# events_features_features_1 = events_features.drop(accounts_ids_not_in_users)

In [15]:
print(f"account shape: {accounts_features.shape}")
print(f"events shape: {events_features.shape}")
print(f"users shape: {users_features.shape}")

account shape: (716828, 7)
events shape: (716628, 22)
users shape: (716099, 4)


In [16]:
accounts_ids_not_in_users = accounts_features[
    ~accounts_features.index.isin(users_features.index)
]
accounts_ids_not_in_events = accounts_features[
    ~accounts_features.index.isin(events_features.index)
]

In [17]:
in_users_not_in_accounts = users_features[
    ~users_features.index.isin(account_features.index)
]
in_events_not_in_accounts = events_features[
    ~events_features.index.isin(account_features.index)
]

In [18]:
in_events_not_in_accounts

,active_of_users,active_days,total_events,column_events,board_events,num_of_boards,count_kind_columns,content_events,group_events,invite_events,...,new_entry_events,payment_events,inbox_events,communicating_events,non_communicating_events,web_events,ios_events,android_events,desktop_app_events,empty_events
account_id,,,,,,,,,,,,,,,,,,,,,


In [19]:
features = accounts_features.merge(
    users_features,
    left_index=True,
    right_index=True,
    how="outer",
).merge(
    events_features,
    left_index=True,
    right_index=True,
    how="outer",
)
target = features.pop("lead_score")

In [20]:
features.head()

,paying,collection_21_days,max_team_size,min_team_size,industry,payment_currency,registered_users,number_of_admins,number_of_pending_users,number_of_enabled_users,...,new_entry_events,payment_events,inbox_events,communicating_events,non_communicating_events,web_events,ios_events,android_events,desktop_app_events,empty_events
account_id,,,,,,,,,,,,,,,,,,,,,
4,0,0,NaN,NaN,NaN,USD,1.0,1.0,0.0,1.0,...,2.0,0.0,0.0,0.0,2.0,0.0,0.0,42.0,0.0,7.0
6,0,0,5.0,2.0,Finance,USD,1.0,1.0,0.0,1.0,...,1.0,0.0,1.0,1.0,2.0,56.0,0.0,0.0,0.0,7.0
7,0,0,1.0,1.0,Business Owner,USD,1.0,1.0,0.0,1.0,...,1.0,0.0,1.0,1.0,75.0,292.0,0.0,0.0,0.0,17.0
8,0,0,NaN,NaN,NaN,USD,1.0,1.0,0.0,1.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,19.0,0.0,3.0
9,0,0,0.0,0.0,NaN,USD,1.0,1.0,0.0,1.0,...,3.0,0.0,3.0,1.0,6.0,155.0,92.0,0.0,0.0,21.0


In [22]:
features

duplicate_mask = features.duplicated()

# Select duplicate rows based on the mask
duplicate_rows = features[duplicate_mask]
duplicate_rows

,paying,collection_21_days,max_team_size,min_team_size,industry,payment_currency,registered_users,number_of_admins,number_of_pending_users,number_of_enabled_users,...,new_entry_events,payment_events,inbox_events,communicating_events,non_communicating_events,web_events,ios_events,android_events,desktop_app_events,empty_events
account_id,,,,,,,,,,,,,,,,,,,,,
799,0,0,NaN,NaN,NaN,EUR,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
828,0,0,5.0,2.0,NaN,EUR,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
908,0,0,5.0,2.0,NaN,EUR,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
956,0,0,5.0,2.0,NaN,USD,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
969,0,0,1.0,1.0,NaN,USD,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1433446,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1433471,0,0,1.0,1.0,NaN,USD,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1433535,0,0,15.0,11.0,NaN,USD,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
